In [52]:
%cd thesis
import sys

sys.path.append(os.getcwd())

[Errno 2] No such file or directory: 'thesis'
/home/jupyter/work/resources/thesis


In [143]:
%pip install -r requirements.conda.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [26]:
import torch
import os
from torchsummary import summary
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [48]:
torch.cuda.is_available()

False

In [55]:
# Upsampler

model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64)

upsampler = RealESRGANer(
    scale=4,
    model_path='./realesrgan/experiments/pretrained_models/RealESRGAN_x4plus.pth',
    model=model,
    tile=False,
    tile_pad=10,
    pre_pad=0,
    half=False)

FileNotFoundError: [Errno 2] No such file or directory: './realesrgan/experiments/pretrained_models/RealESRGAN_x4plus.pth'

In [50]:
from src.helpers import utils
from compress import make_deterministic
from src.loss.perceptual_similarity import perceptual_loss as ps
from default_config import ModelModes

In [57]:
# Compression

# Reproducibility
make_deterministic()
perceptual_loss_fn = ps.PerceptualLoss(model='net-lin', net='alex', use_gpu=torch.cuda.is_available())

# Load model
device = torch.device('cpu')#utils.get_device()
print(device)
logger = utils.logger_setup(logpath=os.path.join('images', 'logs'), filepath=os.path.abspath('1'))
loaded_args, compression, _ = utils.load_model('experiments/hific_low.pt', logger, device, model_mode=ModelModes.EVALUATION,
    current_args_d=None, prediction=True, strict=False)
print('logger done')

#compression.Hyperprior.hyperprior_entropy_model.build_tables()

17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:32:56 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
100%|██████████| 64/64 [00:00<00:00, 356.12it/s]
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: Loading model ...
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL TYPE: compression_gan
17:35:00 INFO - load_model: MODEL MODE: evaluation
17:35:00 INFO - load_model: MODEL MODE: evaluation

Setting up Perceptual loss...
Loading model from: /home/jupyter/work/resources/thesis/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done
cpu
Building prior probability tables...
Setting up Perceptual loss...
Loading model from: /home/jupyter/work/resources/thesis/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done
logger done


In [59]:
# Input size: [2, 220, 8, 8]

input = torch.rand(1, 220, 8, 8)

result = compression.Generator(input)
summary(compression.Generator, (220, 8, 8), 1, device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
     ChannelNorm2D-1             [1, 220, 8, 8]               0
   ReflectionPad2d-2           [1, 220, 10, 10]               0
   ReflectionPad2d-3           [1, 220, 10, 10]               0
            Conv2d-4             [1, 960, 8, 8]       1,901,760
     ChannelNorm2D-5             [1, 960, 8, 8]               0
   ReflectionPad2d-6           [1, 960, 10, 10]               0
            Conv2d-7             [1, 960, 8, 8]       8,295,360
     ChannelNorm2D-8             [1, 960, 8, 8]               0
   ReflectionPad2d-9           [1, 960, 10, 10]               0
           Conv2d-10             [1, 960, 8, 8]       8,295,360
    ChannelNorm2D-11             [1, 960, 8, 8]               0
    ResidualBlock-12             [1, 960, 8, 8]               0
  ReflectionPad2d-13           [1, 960, 10, 10]               0
           Conv2d-14             [1, 96

In [60]:
summary(compression, (3, 128, 128), 2, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ReflectionPad2d-1           [2, 3, 134, 134]               0
   ReflectionPad2d-2           [2, 3, 134, 134]               0
            Conv2d-3          [2, 60, 128, 128]           8,880
     ChannelNorm2D-4          [2, 60, 128, 128]               0
              ReLU-5          [2, 60, 128, 128]               0
   ReflectionPad2d-6          [2, 60, 129, 129]               0
   ReflectionPad2d-7          [2, 60, 129, 129]               0
   ReflectionPad2d-8          [2, 60, 129, 129]               0
   ReflectionPad2d-9          [2, 60, 129, 129]               0
  ReflectionPad2d-10          [2, 60, 129, 129]               0
           Conv2d-11           [2, 120, 64, 64]          64,920
    ChannelNorm2D-12           [2, 120, 64, 64]               0
             ReLU-13           [2, 120, 64, 64]               0
  ReflectionPad2d-14           [2, 120,

/kernel/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


TypeError: can't multiply sequence by non-int of type 'list'

In [61]:
summary(upsampler.model, (3, 128, 128), 1, device='cpu')

NameError: name 'upsampler' is not defined

In [62]:
compression.train(False)
compression(torch.rand((16, 3, 128, 128)))

(tensor([[[[0.7440, 0.7595, 0.7649,  ..., 0.7502, 0.7729, 0.7499],
           [0.7460, 0.7439, 0.7479,  ..., 0.7267, 0.7630, 0.7519],
           [0.7240, 0.7518, 0.7720,  ..., 0.7460, 0.7700, 0.7553],
           ...,
           [0.7602, 0.7695, 0.7524,  ..., 0.7332, 0.7431, 0.7577],
           [0.7697, 0.7790, 0.7673,  ..., 0.7373, 0.7333, 0.7527],
           [0.7766, 0.7952, 0.7854,  ..., 0.7635, 0.7356, 0.7380]],
 
          [[0.7531, 0.7618, 0.7628,  ..., 0.7473, 0.7645, 0.7329],
           [0.7456, 0.7349, 0.7376,  ..., 0.7210, 0.7521, 0.7351],
           [0.7248, 0.7477, 0.7639,  ..., 0.7524, 0.7655, 0.7395],
           ...,
           [0.7719, 0.7779, 0.7592,  ..., 0.7340, 0.7433, 0.7556],
           [0.7761, 0.7823, 0.7676,  ..., 0.7410, 0.7357, 0.7511],
           [0.7770, 0.7948, 0.7889,  ..., 0.7655, 0.7365, 0.7308]],
 
          [[0.7570, 0.7779, 0.7804,  ..., 0.7727, 0.7882, 0.7655],
           [0.7579, 0.7613, 0.7643,  ..., 0.7439, 0.7765, 0.7695],
           [0.7298, 0.76

In [63]:
class Model(torch.nn.Module):
    def __init__(self, compression, upsampler):
        super().__init__()
        self.compression = compression
        self.upsampler = upsampler
    
    def forward(self, x):
        y, loss = self.compression(x)
        y = self.upsampler(y)
        return y

    def train(self, train=True):
        self.compression.train(False)
        self.compression.Generator.train(train)
        self.upsampler.train(train)

In [64]:
m = Model(compression, upsampler.model)

NameError: name 'upsampler' is not defined

In [ ]:
x = torch.rand(1, 3, 128, 128)
result = m(x)

print(result.shape)

### Remarks
- Conv2D with 2x2 padding that's equivalent to Conv2DTranspose with no padding.
- Checkerboard artifacts can start to become an issue when using strides (even after stacking multiple layers).

### Things to try: 

1. To avoid checkerboard artifacts, an alternative upsampling method that’s gaining popularity is to apply classical upsampling followed by a regular convolution (that preserves the spatial dimensions).

In [67]:
import torch
import torch.nn.functional as F
from src.normalisation import channel, instance

class ResidualBlock(torch.nn.Module):
    def __init__(self, input_dims, kernel_size=3, stride=1, 
                 channel_norm=True, activation='relu'):
        """
        input_dims: Dimension of input tensor (B,C,H,W)
        """
        super(ResidualBlock, self).__init__()

        self.activation = getattr(F, activation)
        in_channels = input_dims[1]
        norm_kwargs = dict(momentum=0.1, affine=True, track_running_stats=False)

        if channel_norm is True:
            self.interlayer_norm = channel.ChannelNorm2D_wrap
        else:
            self.interlayer_norm = instance.InstanceNorm2D_wrap

        pad_size = int((kernel_size-1)/2)
        self.pad = torch.nn.ReflectionPad2d(pad_size)
        self.conv1 = torch.nn.Conv2d(in_channels, in_channels, kernel_size, stride=stride)
        self.conv2 = torch.nn.Conv2d(in_channels, in_channels, kernel_size, stride=stride)
        self.norm1 = self.interlayer_norm(in_channels, **norm_kwargs)
        self.norm2 = self.interlayer_norm(in_channels, **norm_kwargs)

    def forward(self, x):
        identity_map = x
        res = self.pad(x)
        res = self.conv1(res)
        res = self.norm1(res) 
        res = self.activation(res)

        res = self.pad(res)
        res = self.conv2(res)
        res = self.norm2(res)

        return torch.add(res, identity_map)

class Upsampler(torch.nn.Module):

    def __init__(self, input_dims, batch_size, C=220, activation='relu',
                 n_residual_blocks=8, channel_norm=True, sample_noise=False,
                 noise_dim=32, silent=True):
        super(Upsampler, self).__init__()
        self.silent = silent

        kernel_dim = 3
        filters = [960, 480, 240, 120, 60]
        self.n_residual_blocks = n_residual_blocks
        self.sample_noise = sample_noise
        self.noise_dim = noise_dim

        # Layer / normalization options
        cnn_kwargs = dict(stride=2, padding=1, output_padding=1)
        norm_kwargs = dict(momentum=0.1, affine=True, track_running_stats=False)
        activation_d = dict(relu='ReLU', elu='ELU', leaky_relu='LeakyReLU')
        self.activation = getattr(torch.nn, activation_d[activation])  # (leaky_relu, relu, elu)
        self.n_upsampling_layers = 4
        
        if channel_norm is True:
            self.interlayer_norm = channel.ChannelNorm2D_wrap
        else:
            self.interlayer_norm = instance.InstanceNorm2D_wrap

        self.pre_pad = torch.nn.ReflectionPad2d(1)
        self.asymmetric_pad = torch.nn.ReflectionPad2d((0,1,1,0))  # Slower than tensorflow?
        self.post_pad = torch.nn.ReflectionPad2d(3)

        H0, W0 = input_dims[1:]
        heights = [2**i for i in range(5,9)]
        widths = heights
        H1, H2, H3, H4 = heights
        W1, W2, W3, W4 = widths 


        # (16,16) -> (16,16), with implicit padding
        self.conv_block_init = torch.nn.Sequential(
            self.interlayer_norm(C, **norm_kwargs),
            self.pre_pad,
            torch.nn.Conv2d(C, filters[0], kernel_size=(3,3), stride=1),
            self.interlayer_norm(filters[0], **norm_kwargs),
        )

        if sample_noise is True:
            # Concat noise with latent representation
            filters[0] += self.noise_dim

        for m in range(n_residual_blocks):
            resblock_m = ResidualBlock(input_dims=(batch_size, filters[0], H0, W0), 
                channel_norm=channel_norm, activation=activation)
            self.add_module(f'resblock_{str(m)}', resblock_m)
        
        self.upconv_block1 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(filters[0], filters[1], kernel_dim, **cnn_kwargs),
            self.interlayer_norm(filters[1], **norm_kwargs),
            self.activation(),
        )
        self.upconv_block2 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(filters[1], filters[2], kernel_dim, **cnn_kwargs),
            self.interlayer_norm(filters[2], **norm_kwargs),
            self.activation(),
        )
        self.upconv_block3 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(filters[2], filters[3], kernel_dim, **cnn_kwargs),
            self.interlayer_norm(filters[3], **norm_kwargs),
            self.activation(),
        )
        self.upconv_block4 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(filters[3], filters[4], kernel_dim, **cnn_kwargs),
            self.interlayer_norm(filters[4], **norm_kwargs),
            self.activation(),
        )
        self.conv_block_out = torch.nn.Sequential(
            self.post_pad,
            torch.nn.Conv2d(filters[-1], 3, kernel_size=(7,7), stride=1),
        )

    def forward(self, x):
        if not self.silent:
            print(f'{"INPUT : ": <15}', x.shape)
        head = self.conv_block_init(x)
        if not self.silent:
            print(f'{"INITIAL_CONV : ": <15}', head.shape)

        if self.sample_noise is True:
            B, C, H, W = tuple(head.size())
            z = torch.randn((B, self.noise_dim, H, W)).to(head)
            head = torch.cat((head,z), dim=1)

        for m in range(self.n_residual_blocks):
            resblock_m = getattr(self, f'resblock_{str(m)}')
            if m == 0:
                x = resblock_m(head)
            else:
                x = resblock_m(x)
            if not self.silent:
                print(f'{f"RESIDUAL_{m}": <15}', x.shape)
        
        x += head
        x = self.upconv_block1(x)
        if not self.silent:
            print(f'{"UPCONV_1 : ": <15}', x.shape)
        x = self.upconv_block2(x)
        if not self.silent:
            print(f'{"UPCONV_2 : ": <15}', x.shape)
        x = self.upconv_block3(x)
        if not self.silent:
            print(f'{"UPCONV_3 : ": <15}', x.shape)
        x = torch.nn.functional.interpolate(x, scale_factor=2, mode='nearest')
        if not self.silent:
            print(f'{"BILINEAR : ": <15}', x.shape)
        x = self.upconv_block4(x)
        if not self.silent:
            print(f'{"UPCONV_4 : ": <15}', x.shape)
        out = self.conv_block_out(x)
        if not self.silent:
            print(f'{"UPCONV_5 : ": <15}', out.shape)

        return out

In [ ]:
upsampler = Upsampler([220, 8, 8], 2)

input = torch.rand(2, 220, 8, 8)
output = upsampler(input)

print(output.shape)

In [ ]:
summary(upsampler, (220, 8, 8), 2, device='cpu')

Try using interpolation to upscale feature maps. Need to match shapes.

In [ ]:
import torch
from torch.nn import functional as F

shape = (2, 220, 8, 8)

input = torch.rand(shape)
output1 = F.interpolate(input, scale_factor=2, mode='nearest')

print(input.shape, output1.shape)

### Load model from checkpoint and modify structure of Generator

In [65]:
# Compression
import torch
from src.helpers import utils
from compress import make_deterministic
from src.loss.perceptual_similarity import perceptual_loss as ps
from default_config import ModelModes

# Reproducibility
make_deterministic()
perceptual_loss_fn = ps.PerceptualLoss(model='net-lin', net='alex', use_gpu=torch.cuda.is_available())

# Load model
device = torch.device('cpu')#utils.get_device()
logger = utils.logger_setup(logpath=os.path.join('images', 'logs'), filepath=os.path.abspath('1'))
loaded_args, compression, _ = utils.load_model('experiments/hific_low.pt', logger, device, model_mode=ModelModes.EVALUATION,
    current_args_d=None, prediction=True, strict=False, silent=True)

#compression.Hyperprior.hyperprior_entropy_model.build_tables()

17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
17:36:52 INFO - logger_setup: /home/jupyter/work/resources/thesis/1
100%|██████████| 64/64 [00:00<00:00, 458.19it/s]
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Loading model ...
17:36:55 INFO - load_model: Estimated model size (under fp32): 593.146 MB
17:36:55 INFO - load_model: Estimated model size (under fp32): 593.146 MB
17:36:55 INFO - load_model: Estimated model size (under fp32): 593.146 MB
17:36:55 INFO - load_model: Estimated model s

Setting up Perceptual loss...
Loading model from: /home/jupyter/work/resources/thesis/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done
Building prior probability tables...
Setting up Perceptual loss...
Loading model from: /home/jupyter/work/resources/thesis/src/loss/perceptual_similarity/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


In [68]:
upsampler = Upsampler((220, 8, 8), 1)

In [69]:
compression.train(False)
compression.Generator = upsampler
compression.Generator.train(True)

Upsampler(
  (pre_pad): ReflectionPad2d((1, 1, 1, 1))
  (asymmetric_pad): ReflectionPad2d((0, 1, 1, 0))
  (post_pad): ReflectionPad2d((3, 3, 3, 3))
  (conv_block_init): Sequential(
    (0): ChannelNorm2D()
    (1): ReflectionPad2d((1, 1, 1, 1))
    (2): Conv2d(220, 960, kernel_size=(3, 3), stride=(1, 1))
    (3): ChannelNorm2D()
  )
  (resblock_0): ResidualBlock(
    (pad): ReflectionPad2d((1, 1, 1, 1))
    (conv1): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))
    (norm1): ChannelNorm2D()
    (norm2): ChannelNorm2D()
  )
  (resblock_1): ResidualBlock(
    (pad): ReflectionPad2d((1, 1, 1, 1))
    (conv1): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))
    (norm1): ChannelNorm2D()
    (norm2): ChannelNorm2D()
  )
  (resblock_2): ResidualBlock(
    (pad): ReflectionPad2d((1, 1, 1, 1))
    (conv1): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1))


In [70]:
input = torch.rand((1, 3, 128, 128))
output = compression(input)

print(f'OUTPUT SHAPE : {output[0].shape}')

OUTPUT SHAPE : torch.Size([1, 3, 128, 128])


### Now we got to train the network

For this purpose we need:

1. Set up a dataset
2. Run training script from hific repo

In [104]:
%pip install awscli

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [108]:
%pip install boto3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 6.2 MB 1.2 MB/s            
     |████████████████████████████████| 73 kB 1.5 MB/s             
  Attempting uninstall: botocore
    Found existing installation: botocore 1.23.47
    Uninstalling botocore-1.23.47:
      Successfully uninstalled botocore-1.23.47
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.5.1
    Uninstalling s3transfer-0.5.1:
      Successfully uninstalled s3transfer-0.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.47 requires botocore==1.23.47, but you have botocore 1.15.49 which is incompatible.
awscli 1.22.47 requires s3transfer<0.6.0,>=0.5.0, but you have s3transfer 0.3.7 which is incompatible.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 2.10 which is incompati

In [148]:
#pragma dataset init OPEN_IMAGES --size 128Gb

# TODO: fill dataset here
import boto3
from botocore.config import Config
from botocore import UNSIGNED

import tqdm

import tarfile

BUCKET = 'open-images-dataset'
KEY = 'tar/train_0.tar.gz'

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
metadata = s3.head_object(Bucket=BUCKET, Key=KEY)

with tqdm.tqdm(total=metadata['ContentLength'], unit="B", unit_scale=True) as pbar:
    s3.download_file(BUCKET, KEY, f'{DATASET_PATH}/train_0.tar.gz', Callback=lambda bytes_transfered: pbar.update(bytes_transfered))

print('Unpacking...')

tar = tarfile.open(f'{DATASET_PATH}/train_0.tar.gz')
tar.extractall(f'{DATASET_PATH}')
tar.close()
#!tar -xzf /home/jupyter/mnt/datasets/OPEN_IMAGES/train_0.tar.gz

print('Done.')

# Dataset will be created in /home/jupyter/mnt/datasets/OPEN_IMAGES

It's init dataset task. State result won't be merged.


100%|██████████| 49.3G/49.3G [16:43<00:00, 49.1MB/s]
/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:872: UserWarning: The following variables cannot be serialized: pbar, s3, tar
  warnings.warn(message)


Unpacking...
Done.
Remounting dataset OPEN_IMAGES... ok
Mounting dataset OPEN_IMAGES... /home/jupyter/mnt/datasets/OPEN_IMAGES


Internal error: Something went wrong

In [146]:
DATASET_PATH = '/home/jupyter/mnt/datasets/OPEN_IMAGES'
HOME_PATH = '/home/jupyter/work/resources/thesis'

# /home/jupyter/work/resources/thesis/experiments/hific_v0.1_openimages_compression_2022_02_07_13_42/tensorboard

In [152]:

import numpy as np
import os, time, datetime
import pickle, argparse
import itertools

from tqdm import tqdm, trange
from collections import defaultdict
from attrdict import AttrDict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# Custom modules
from src.model import Model
from src.helpers import utils, datasets
from default_config import hific_args, mse_lpips_args, directories, ModelModes, ModelTypes

# go fast boi!!
#torch.backends.cudnn.benchmark = True

def create_model(args, device, logger, storage, storage_test):

    start_time = time.time()
    model = Model(args, logger, storage, storage_test, model_type=args.model_type)
    logger.info(model)
    logger.info('Trainable parameters:')

    for n, p in model.named_parameters():
        logger.info('{} - {}'.format(n, p.shape))

    logger.info("Number of trainable parameters: {}".format(utils.count_parameters(model)))
    logger.info("Estimated size (under fp32): {:.3f} MB".format(utils.count_parameters(model) * 4. / 10**6))
    logger.info('Model init {:.3f}s'.format(time.time() - start_time))

    return model

def optimize_loss(loss, opt, retain_graph=False):
    loss.backward(retain_graph=retain_graph)
    opt.step()
    opt.zero_grad()

def optimize_compression_loss(compression_loss, amortization_opt, hyperlatent_likelihood_opt):
    compression_loss.backward()
    amortization_opt.step()
    hyperlatent_likelihood_opt.step()
    amortization_opt.zero_grad()
    hyperlatent_likelihood_opt.zero_grad()

def test(args, model, epoch, idx, data, test_data, test_bpp, device, epoch_test_loss, storage, best_test_loss, 
         start_time, epoch_start_time, logger, train_writer, test_writer):

    model.eval()  
    with torch.no_grad():
        data = data.to(device, dtype=torch.float)

        losses, intermediates = model(data, return_intermediates=True, writeout=False)
        utils.save_images(train_writer, model.step_counter, intermediates.input_image, intermediates.reconstruction,
            fname=os.path.join(args.figures_save, 'recon_epoch{}_idx{}_TRAIN_{:%Y_%m_%d_%H:%M}.jpg'.format(epoch, idx, datetime.datetime.now())))

        test_data = test_data.to(device, dtype=torch.float)
        losses, intermediates = model(test_data, return_intermediates=True, writeout=True)
        utils.save_images(test_writer, model.step_counter, intermediates.input_image, intermediates.reconstruction,
            fname=os.path.join(args.figures_save, 'recon_epoch{}_idx{}_TEST_{:%Y_%m_%d_%H:%M}.jpg'.format(epoch, idx, datetime.datetime.now())))
    
        compression_loss = losses['compression'] 
        epoch_test_loss.append(compression_loss.item())
        mean_test_loss = np.mean(epoch_test_loss)
        
        best_test_loss = utils.log(model, storage, epoch, idx, mean_test_loss, compression_loss.item(), 
                                     best_test_loss, start_time, epoch_start_time, 
                                     batch_size=data.shape[0], avg_bpp=test_bpp.mean().item(),header='[TEST]', 
                                     logger=logger, writer=test_writer)
        
    return best_test_loss, epoch_test_loss


def train(args, model, train_loader, test_loader, device, logger, optimizers):

    start_time = time.time()
    test_loader_iter = iter(test_loader)
    current_D_steps, train_generator = 0, True
    best_loss, best_test_loss, mean_epoch_loss = np.inf, np.inf, np.inf     
    train_writer = SummaryWriter(os.path.join(args.tensorboard_runs, 'train'))
    test_writer = SummaryWriter(os.path.join(args.tensorboard_runs, 'test'))
    storage, storage_test = model.storage_train, model.storage_test

    amortization_opt, hyperlatent_likelihood_opt = optimizers['amort'], optimizers['hyper']
    if model.use_discriminator is True:
        disc_opt = optimizers['disc']

    for epoch in trange(args.n_epochs, desc='Epoch'):

        epoch_loss, epoch_test_loss = [], []  
        epoch_start_time = time.time()
        
        if epoch > 0:
            ckpt_path = utils.save_model(model, optimizers, mean_epoch_loss, epoch, device, args=args, logger=logger)
        
        model.train()

        for idx, (data, bpp) in enumerate(tqdm(train_loader, desc='Train'), 0):

            data = data.to(device, dtype=torch.float)
            
            try:
                if model.use_discriminator is True:
                    # Train D for D_steps, then G, using distinct batches
                    losses = model(data, train_generator=train_generator)
                    compression_loss = losses['compression']
                    disc_loss = losses['disc']

                    if train_generator is True:
                        optimize_compression_loss(compression_loss, amortization_opt, hyperlatent_likelihood_opt)
                        train_generator = False
                    else:
                        optimize_loss(disc_loss, disc_opt)
                        current_D_steps += 1

                        if current_D_steps == args.discriminator_steps:
                            current_D_steps = 0
                            train_generator = True

                        continue
                else:
                    # Rate, distortion, perceptual only
                    losses = model(data, train_generator=True)
                    compression_loss = losses['compression']
                    optimize_compression_loss(compression_loss, amortization_opt, hyperlatent_likelihood_opt)

            except KeyboardInterrupt:
                # Note: saving not guaranteed!
                if model.step_counter > args.log_interval+1:
                    logger.warning('Exiting, saving ...')
                    ckpt_path = utils.save_model(model, optimizers, mean_epoch_loss, epoch, device, args=args, logger=logger)
                    return model, ckpt_path
                else:
                    return model, None

            if model.step_counter % args.log_interval == 1:
                epoch_loss.append(compression_loss.item())
                mean_epoch_loss = np.mean(epoch_loss)

                best_loss = utils.log(model, storage, epoch, idx, mean_epoch_loss, compression_loss.item(),
                                best_loss, start_time, epoch_start_time, batch_size=data.shape[0],
                                avg_bpp=bpp.mean().item(), logger=logger, writer=train_writer)
                try:
                    test_data, test_bpp = test_loader_iter.next()
                except StopIteration:
                    test_loader_iter = iter(test_loader)
                    test_data, test_bpp = test_loader_iter.next()

                best_test_loss, epoch_test_loss = test(args, model, epoch, idx, data, test_data, test_bpp, device, epoch_test_loss, storage_test,
                     best_test_loss, start_time, epoch_start_time, logger, train_writer, test_writer)

                with open(os.path.join(args.storage_save, 'storage_{}_tmp.pkl'.format(args.name)), 'wb') as handle:
                    pickle.dump(storage, handle, protocol=pickle.HIGHEST_PROTOCOL)

                model.train()

                # LR scheduling
                utils.update_lr(args, amortization_opt, model.step_counter, logger)
                utils.update_lr(args, hyperlatent_likelihood_opt, model.step_counter, logger)
                if model.use_discriminator is True:
                    utils.update_lr(args, disc_opt, model.step_counter, logger)

                if model.step_counter > args.n_steps:
                    logger.info('Reached step limit [args.n_steps = {}]'.format(args.n_steps))
                    break

            if (idx % args.save_interval == 1) and (idx > args.save_interval):
                ckpt_path = utils.save_model(model, optimizers, mean_epoch_loss, epoch, device, args=args, logger=logger)

        # End epoch
        mean_epoch_loss = np.mean(epoch_loss)
        mean_epoch_test_loss = np.mean(epoch_test_loss)

        logger.info('===>> Epoch {} | Mean train loss: {:.3f} | Mean test loss: {:.3f}'.format(epoch, 
            mean_epoch_loss, mean_epoch_test_loss))    

        if model.step_counter > args.n_steps:
            break
    
    with open(os.path.join(args.storage_save, 'storage_{}_{:%Y_%m_%d_%H:%M:%S}.pkl'.format(args.name, datetime.datetime.now())), 'wb') as handle:
        pickle.dump(storage, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    ckpt_path = utils.save_model(model, optimizers, mean_epoch_loss, epoch, device, args=args, logger=logger)
    args.ckpt = ckpt_path
    logger.info("Training complete. Time elapsed: {:.3f} s. Number of steps: {}".format((time.time()-start_time), model.step_counter))
    
    return model, ckpt_path


def run(generator: torch.nn.Module = None):
    cmd_args = AttrDict({
    # General options - see `default_config.py` for full options
        "name": None,
        "model_type": ModelTypes.COMPRESSION,
        "regime": 'low',
        "gpu": 0,
        "force_set_gpu": False,
        "log_interval": hific_args.log_interval,
        "save_interval": hific_args.save_interval,
        "multigpu": False,
        "normalize_input_image": False,
        "batch_size": hific_args.batch_size,
        "save": "experiments",
        "likelihood_type": 'gaussian',
        "force_set_gpu": False,
        "use_latent_mixture_model": False,
        "n_steps": hific_args.n_steps,
        "n_epochs": hific_args.n_epochs, 
        "learning_rate": hific_args.learning_rate,
        "weight_decay": hific_args.weight_decay,
        "latent_channels": hific_args.latent_channels,
        "n_residual_blocks": hific_args.n_residual_blocks,
        "warmstart": False,
        "warmstart_ckpt": None,
    })

    if (cmd_args.gpu != 0) or (cmd_args.force_set_gpu is True):
        torch.cuda.set_device(cmd_args.gpu)

    if cmd_args.model_type == ModelTypes.COMPRESSION:
        args = mse_lpips_args
    elif cmd_args.model_type == ModelTypes.COMPRESSION_GAN:
        args = hific_args

    start_time = time.time()
    device = utils.get_device()

    # Override default arguments from config file with provided command line arguments
    dictify = lambda x: dict((n, getattr(x, n)) for n in dir(x) if not (n.startswith('__') or 'logger' in n))
    args_d, cmd_args_d = dictify(args), vars(cmd_args)
    args_d.update(cmd_args_d)
    args = utils.Struct(**args_d)
    args = utils.setup_generic_signature(args, special_info=args.model_type)
    args.target_rate = args.target_rate_map[args.regime]
    args.lambda_A = args.lambda_A_map[args.regime]
    args.n_steps = int(args.n_steps)
    args.warmstart = cmd_args.warmstart

    storage = defaultdict(list)
    storage_test = defaultdict(list)
    logger = utils.logger_setup(logpath=os.path.join(args.snapshot, 'logs'), filepath=os.getcwd())

    if args.warmstart is True:
        assert args.warmstart_ckpt is not None, 'Must provide checkpoint to previously trained AE/HP model.'
        logger.info('Warmstarting discriminator/generator from autoencoder/hyperprior model.')
        if args.model_type != ModelTypes.COMPRESSION_GAN:
            logger.warning('Should warmstart compression-gan model.')
        # TODO : Define a custom model type and pass it through these frunctions
        args, model, optimizers = utils.load_model(args.warmstart_ckpt, logger, device, 
            model_type=args.model_type, current_args_d=dictify(args), strict=False, prediction=False)
    else:
        model = create_model(args, device, logger, storage, storage_test)

        # In case we are starting training, attach custom generator here
        if generator:
            model.Generator = generator

        model = model.to(device)
        amortization_parameters = itertools.chain.from_iterable(
            [am.parameters() for am in model.amortization_models])

        hyperlatent_likelihood_parameters = model.Hyperprior.hyperlatent_likelihood.parameters()

        amortization_opt = torch.optim.Adam(amortization_parameters,
            lr=args.learning_rate)
        hyperlatent_likelihood_opt = torch.optim.Adam(hyperlatent_likelihood_parameters, 
            lr=args.learning_rate)
        optimizers = dict(amort=amortization_opt, hyper=hyperlatent_likelihood_opt)

        if model.use_discriminator is True:
            discriminator_parameters = model.Discriminator.parameters()
            disc_opt = torch.optim.Adam(discriminator_parameters, lr=args.learning_rate)
            optimizers['disc'] = disc_opt

    n_gpus = torch.cuda.device_count()
    if n_gpus > 1 and args.multigpu is True:
        # Not supported at this time
        raise NotImplementedError('MultiGPU not supported yet.')
        logger.info('Using {} GPUs.'.format(n_gpus))
        model = nn.DataParallel(model)

    logger.info('MODEL TYPE: {}'.format(args.model_type))
    logger.info('MODEL MODE: {}'.format(args.model_mode))
    logger.info('BITRATE REGIME: {}'.format(args.regime))
    logger.info('SAVING LOGS/CHECKPOINTS/RECORDS TO {}'.format(args.snapshot))
    logger.info('USING DEVICE {}'.format(device))
    logger.info('USING GPU ID {}'.format(args.gpu))
    logger.info('USING DATASET: {}'.format(args.dataset))

    test_loader = datasets.get_dataloaders(args.dataset,
                                root=args.dataset_path,
                                batch_size=args.batch_size,
                                logger=logger,
                                mode='validation',
                                shuffle=True,
                                normalize=args.normalize_input_image)

    train_loader = datasets.get_dataloaders(args.dataset,
                                root=args.dataset_path,
                                batch_size=args.batch_size,
                                logger=logger,
                                mode='train',
                                shuffle=True,
                                normalize=args.normalize_input_image)

    args.n_data = len(train_loader.dataset)
    args.image_dims = train_loader.dataset.image_dims
    logger.info('Training elements: {}'.format(args.n_data))
    logger.info('Input Dimensions: {}'.format(args.image_dims))
    logger.info('Optimizers: {}'.format(optimizers))
    logger.info('Using device {}'.format(device))

    metadata = dict((n, getattr(args, n)) for n in dir(args) if not (n.startswith('__') or 'logger' in n))
    logger.info(metadata)

    """
    Train
    """
    model, ckpt_path = train(args, model, train_loader, test_loader, device, logger, optimizers=optimizers)

    """
    TODO
    Generate metrics
    """


In [ ]:
#!g2.mig
run()

In [170]:
%tensorboard --logdir /home/jupyter/work/resources/thesis/experiments/hific_v0.1_openimages_compression_2022_02_07_13_42

Launching TensorBoard...
